## Policy Gradient
$$
\nabla_\theta J(\theta)  = \mathbb{E}_{s \sim d_\pi}\left [\mathbb{E}_{a \sim \pi_{\theta}(\cdot \vert s)} [Q_\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)] \right ] 
$$
Sample from $a \sim \pi_{\theta}(\cdot \vert s)$ and a monte carlo estimation of $\nabla_\theta J(\theta)$ is:
$$
\boldsymbol{g}_\theta(s, a) \doteq Q_{\pi_\theta}(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)
$$

## Generalization Advantage Estimation(GAE)
Substitute $Q_\pi(s, a)$ with $A_\pi(s, a)$, the advantage function:
$$
\boldsymbol{g}_\theta(s, a) \doteq A_{\pi_\theta}(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)
$$

## The TRPO formula
Importance sampling ratio:
$$
J^\text{TRPO} (\theta) = \mathbb{E}_{s}\left [\mathbb{E}_{a \sim \pi_{\theta_\text{old}}(\cdot \vert s)} [  \frac{\pi_\theta(a \vert s)}{\pi_{\theta_\text{old}}(a \vert s)} \hat{A}_{\theta_\text{old}}(s, a) ] \right]
$$

## The PPO Algorithm
$$
J^\text{CLIP} (\theta) = \mathbb{E}_{a \sim \pi_{\theta_\text{old}}(\cdot \vert s)} [\min(\frac{\pi_\theta(a \vert s)}{\pi_{\theta_\text{old}}(a \vert s)} \hat{A}_{\theta_\text{old}}(s, a), \text{clip}(r(\theta), 1 - \epsilon, 1 + \epsilon) \hat{A}_{\theta_\text{old}}(s, a))]
$$
